In [1]:
# !unzip -uq "testset_pn.zip" -d "testset_pn/"

In [1]:
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
import glob

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
path_positve='testset_pn/positive' # 기쁨
path_nopositive='testset_pn/non-positive' # 기쁨 아님

path_negative='testset_pn/negative' # 찡그림
path_nonegative='testset_pn/non-negative' # 찡그림 아님


la=len(os.listdir(path_positve))
lb=len(os.listdir(path_nopositive))

lc=len(os.listdir(path_negative))
ld=len(os.listdir(path_nonegative))
# lc=len(os.listdir(path_unkno)) 

print('positive에 저장된 파일의 개수:'+str(la))
print('no positive에 저장된 파일의 개수:'+str(lb))
print('negative에 저장된 파일의 개수:'+str(lc))
print('no negative에 저장된 파일의 개수:'+str(ld))

positive에 저장된 파일의 개수:5861
no positive에 저장된 파일의 개수:5800
negative에 저장된 파일의 개수:5857
no negative에 저장된 파일의 개수:5800


In [4]:
xsize=100
ysize=100

In [5]:
x=[]
y=[]

from keras.preprocessing import image
import numpy as np

def read_dir(path, label):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            
            y.append(label)
            x.append(img_tr)
        except:
            pass

In [6]:
# 기쁨 판정 모델
read_dir(path_positve, 1) 
read_dir(path_nopositive, 0) 

# # 찡그림 판정 모델
# read_dir(path_negative, 1) 
# read_dir(path_nonegative, 0) 

In [43]:
x=np.array(x)
y=np.array(y)

In [44]:
print(x.shape)

(11661, 100, 100, 3)


In [45]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)

input_shape = (xsize,ysize, 3)

# batch_size = 32
# num_classes = 2
# epochs = 15

In [46]:
print(x_test.shape)
print(y_test.shape)

(2333, 100, 100, 3)
(2333,)


In [47]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [ ]:
# cnn2

from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam

# number of possible label values
nb_classes = 2

# Initialising the CNN
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(100, 100, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(nb_classes, activation='softmax'))

opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [50]:
# # 공통 모델
# from keras.optimizers import Adam
# from keras.models import Sequential, Model
# from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
# from keras.applications.mobilenet_v2 import MobileNetV2  
# from keras.applications.mobilenet import MobileNet

# input_shape = (100, 100, 3)
# # base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
# # base_model.trainable=False

# # 미세 조정
# base_model.trainable=True

# set_trainable=False
# for layer in base_model.layers:
#     if layer.name == 'out_relu': # block8~9
#         set_trainable=True
#     if set_trainable:
#         layer.trainable=True
#     else:
#         layer.trainable=False

# x = Flatten()(base_model.output)
# dropout = Dropout(0.9)(x)
# output = Dense(2, activation='softmax', name='predictions')(dropout)
# model = Model(inputs=base_model.input, output=output)

# opt = Adam(lr=0.001) 
# model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy'])

# model.summary()

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 101, 101, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 50, 50, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 50, 50, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`


In [51]:
history = model.fit(x_train, y_train, validation_split=0.5, 
          epochs=20, batch_size=128, verbose=1)

acc = model.evaluate(x_test, y_test)
print(acc)

Train on 4664 samples, validate on 4664 samples
Epoch 1/20
4664/4664 [==============================] - 6s 1ms/step - loss: 2.5072 - accuracy: 0.5965 - val_loss: 1.5834 - val_accuracy: 0.6439
Epoch 2/20
4664/4664 [==============================] - 4s 752us/step - loss: 2.2868 - accuracy: 0.6711 - val_loss: 1.9030 - val_accuracy: 0.6501
Epoch 3/20
4664/4664 [==============================] - 4s 753us/step - loss: 2.1758 - accuracy: 0.6919 - val_loss: 1.6722 - val_accuracy: 0.6784
Epoch 4/20
4664/4664 [==============================] - 4s 754us/step - loss: 2.2593 - accuracy: 0.6861 - val_loss: 2.6009 - val_accuracy: 0.6338
Epoch 5/20
4664/4664 [==============================] - 4s 754us/step - loss: 2.0545 - accuracy: 0.7131 - val_loss: 2.3951 - val_accuracy: 0.6529
Epoch 6/20
4664/4664 [==============================] - 4s 758us/step - loss: 2.2369 - accuracy: 0.7163 - val_loss: 3.1276 - val_accuracy: 0.6364
Epoch 7/20
4664/4664 [==============================] - 4s 755us/step - loss: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('loss, acc')
plt.xlabel('epoch')
plt.ylabel('loss, acc')
plt.legend(['train loss', 'test loss', 'train acc', 'test acc'])
plt.show()

In [70]:
model.save('save/MobileNetV2(positive).h5')
print('모델 저장이 완료되었습니다')

모델 저장이 완료되었습니다


In [71]:
# model.save('save/MobileNetV2(positive).h5')
# print('모델 저장이 완료되었습니다')

In [72]:
# !unzip -uq "roi_1.zip" -d "dataset_pred/"

In [21]:
test_path = "dataset_pred/E01" # 무표정
test_path2 = "dataset_pred/E02" # 기쁨 
test_path3 = "dataset_pred/E03" # 찡그림

X=[]

def read_dir1(path):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            X.append(img_tr)
        except:
            pass

In [22]:
# test set(무표정)
read_dir1(test_path)
X = np.array(X)
pred = model.predict(X)

# 예측(test set 무표정)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 0:
        co+=1

print('\n', "%.2f" % (co/2880.))

0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [23]:
# test set(기쁨)
read_dir1(test_path2)
X = np.array(X)
pred = model.predict(X)

# 예측(test set 기쁨)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 1:
        co+=1

print('\n', "%.2f" % (co/2880.))

0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [24]:
# test set(찡그림)
read_dir1(test_path3)
X = np.array(X)
pred = model.predict(X)

# 예측(test set 찡그림)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 0:
        co+=1

print('\n', "%.2f" % (co/2880.))

0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [ ]:
# E02 데이터셋 정제 후 

# acc : 무표정(0 %), 기쁨(58 %), 찡그림(41 %)